In [159]:
from sklearn import svm
import collections
import numpy as np
import scipy as sc
 

In [21]:
#amino acid sinle letter codes
aac = {}

rest = 'Z';

codes = np.eye(21)
aac['G'] = codes[0]
aac['A'] = codes[1]
aac['L'] = codes[2]
aac['M'] = codes[3]
aac['F'] = codes[4]
aac['W'] = codes[5]
aac['K'] = codes[6]
aac['Q'] = codes[7]
aac['E'] = codes[8]
aac['S'] = codes[9]
aac['P'] = codes[10]
aac['V'] = codes[11]
aac['C'] = codes[12]
aac['I'] = codes[13]
aac['Y'] = codes[14]
aac['H'] = codes[15]
aac['R'] = codes[16]
aac['N'] = codes[17]
aac['D'] = codes[18]
aac['T'] = codes[19]
aac[rest] = codes[20]

alphabeth = ['H', 'E', 'C']
alph = {}
alph['H'] = 0
alph['E'] = 1
alph['C'] = 2

In [160]:
def convert_input(sec, prim, sw):
    length = len(prim)
    ins = []
    outs = []
    for j in range(length - sw):
        inputs = []
        for k in range(j, j+sw):
            if aac.has_key(prim[k]):
                inputs.append(aac[prim[k]])
            else:
                inputs.append(aac[rest])
        ins.append(np.array(inputs).flatten())
        outs.append(alph[sec[j+sw/2]])
    return (ins, outs)

In [167]:
#read dataset file
sw = 9
x = 50

inputs_train = []
outputs_train = []

f = open('dataset.txt', 'r')
for i in range(x):
    f.readline()
    prim = f.readline()
    sec = f.readline()
    ins, outs = convert_input(sec, prim, sw)
    for q in ins:
        inputs_train.append(q)
    for q in outs:
        outputs_train.append(q)
    
f.close()

print len(inputs_train)
print len(outputs_train)

clf = svm.SVC(C=1.5, gamma=0.1)
clf.fit(inputs_train, outputs_train)

9151
9151


SVC(C=1.5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [168]:
def display_result(outputs, alphabet):
    result = ''
    for x in outputs:
        result += alphabet[x]
    return result

def compare(a, b, s):
    l = len(b)
    x = 0.0
    y = 0.0
    for i in range(l - s):
        if a[i] == b[i + s/2]:
            x += 1
        else:
            y += 1
    return x/(x+y)

In [169]:
f = open('dataset.txt', 'r')

z = 50
sum = 0

for i in range(z):
    f.readline()
    prim = f.readline()
    sec = f.readline()
    ins, outs = convert_input(sec, prim, sw)
    pred = display_result(clf.predict(np.array(ins, np.float32)), ['H', 'E', 'C'])
    #print "XXXX" + pred + "XXXX"
    #print sec
    print compare(pred, sec, sw)
    sum += compare(pred, sec, sw)

print "total"
print sum/z

0.91
0.825454545455
0.882352941176
0.901785714286
0.913907284768
0.934362934363
0.905882352941
0.972027972028
0.926470588235
0.870229007634
0.707692307692
0.940594059406
0.821428571429
0.981132075472
0.927203065134
0.917050691244
0.82380952381
0.877906976744
0.885350318471
0.896551724138
0.880281690141
0.900709219858
0.841726618705
0.931623931624
0.947368421053
0.953703703704
0.798076923077
0.754901960784
0.8
0.893617021277
0.938271604938
0.8
0.940298507463
1.0
0.918032786885
0.973684210526
0.876126126126
0.875318066158
0.899470899471
0.886956521739
0.88115942029
0.850609756098
0.942675159236
0.914965986395
0.856088560886
0.887640449438
0.918918918919
0.747035573123
0.955465587045
0.913793103448
total
0.889994267657


In [156]:
def prepare_input_for(sec, prim, sw, struct):
    length = len(prim) - 1
    prefix = ""
    sufix = ""
    for q in range(sw/2):
        prefix += rest
        sufix += rest
    primx = prefix + prim[0:-1] + sufix
    ins = []
    outs = []
    for j in range(length):
        inputs = []
        for k in range(j - sw/2, j + sw/2 + 1):
            if aac.has_key(primx[k]):
                inputs.append(aac[primx[k]])
            else:
                inputs.append(aac[rest])
        ins.append(np.array(inputs).flatten())
        if sec[j] == struct:
            outs.append(1)
        else:
            outs.append(0)
    return (ins, outs)

In [157]:
def prepare_input_for2(sec, prim, sw, sx, sy):
    length = len(prim) - 1
    prefix = ""
    sufix = ""
    for q in range(sw/2):
        prefix += rest
        sufix += rest
    primx = prefix + prim[0:-1] + sufix
    ins = []
    outs = []
    for j in range(length):
        if sec[j] != sx and sec[j] != sy:
            continue
        inputs = []
        for k in range(j - sw/2, j + sw/2 + 1):
            if aac.has_key(primx[k]):
                inputs.append(aac[primx[k]])
            else:
                inputs.append(aac[rest])
        ins.append(np.array(inputs).flatten())
        if sec[j] == sx:
            outs.append(1)
        else:
            outs.append(0)
    return (ins, outs)

In [146]:
def calcQ(pred, sec, struct):
    correct = 1
    observed = 1
    
    for i in range(len(pred)):
        if sec[i] == struct:
            observed += 1
            if sec[i] == pred[i]:
                correct += 1
    
    return 100 * correct / observed

def calcQpred(pred, sec, struct):
    correct = 1
    observed = 1
    
    for i in range(len(pred)):
        if pred[i] == struct:
            observed += 1
            if pred[i] == sec[i]:
                correct += 1
    
    return 100 * correct / observed

In [150]:
inputs_train = []
outputs_train = []

dssp = 'C'
z = 100

f = open('dataset.txt', 'r')
for i in range(z):
    f.readline()
    prim = f.readline()
    sec = f.readline()
    ins, outs = prepare_input_for(sec, prim, sw, dssp)
    for q in ins:
        inputs_train.append(q)
    for q in outs:
        outputs_train.append(q)
    
f.close()

clf_H = svm.SVC(C=1.5, gamma=0.1)
clf_H.fit(inputs_train, outputs_train)

SVC(C=1.5, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [158]:
f = open('dataset.txt', 'r')

for i in range(z+10):
    f.readline()
    prim = f.readline()
    sec = f.readline()

    ins, outs = prepare_input_for(sec, prim, sw, dssp)

    pred = display_result(clf_H.predict(np.array(ins, np.float32)), {0:'X', 1:dssp})
    if i > z:
        print str(calcQ(pred, sec, dssp)) + " : " + str(calcQpred(pred, sec, dssp))

40 : 47
40 : 72
54 : 38
40 : 38
36 : 45
40 : 80
57 : 71
45 : 53
38 : 54
